In [161]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [156]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', required=True, help='cifar10 | lsun | imagenet | folder | lfw | fake')
parser.add_argument('--dataroot', required=True, help='path to dataset')
parser.add_argument('--workers', type=int, help='number of data loading workers', default=2)
parser.add_argument('--batchSize', type=int, default=64, help='input batch size')
parser.add_argument('--imageSize', type=int, default=64, help='the height / width of the input image to network')
parser.add_argument('--nz', type=int, default=100, help='size of the latent z vector')
parser.add_argument('--ngf', type=int, default=64)
parser.add_argument('--ndf', type=int, default=64)
parser.add_argument('--niter', type=int, default=25, help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.0002, help='learning rate, default=0.0002')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for adam. default=0.5')
parser.add_argument('--cuda', action='store_true', help='enables cuda')
parser.add_argument('--ngpu', type=int, default=1, help='number of GPUs to use')
parser.add_argument('--netG', default='', help="path to netG (to continue training)")
parser.add_argument('--netD', default='', help="path to netD (to continue training)")
parser.add_argument('--outf', default='.', help='folder to output images and model checkpoints')
parser.add_argument('--manualSeed', type=int, help='manual seed')

_StoreAction(option_strings=['--manualSeed'], dest='manualSeed', nargs=None, const=None, default=None, type=<type 'int'>, choices=None, help='manual seed', metavar=None)

In [157]:
args = {}

In [202]:
args['batch_size']=100
args['test_batch_size'] =1000

In [203]:
args['manualSeed'] = 5
print("Random Seed: ", args['manualSeed'])

Random Seed:  5


In [204]:
random.seed(args['manualSeed'])
torch.manual_seed(args['manualSeed'])

In [205]:
kwargs = {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['batch_size'], shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['test_batch_size'], shuffle=True, **kwargs)

In [206]:
def _calculate_fan_in_and_fan_out(tensor):
    dimensions = tensor.ndimension()
    if dimensions < 2:
        raise ValueError("Fan in and fan out can not be computed for tensor with less than 2 dimensions")

    if dimensions == 2:  # Linear
        fan_in = tensor.size(1)
        fan_out = tensor.size(0)
    else:
        num_input_fmaps = tensor.size(1)
        num_output_fmaps = tensor.size(0)
        receptive_field_size = 1
        if tensor.dim() > 2:
            receptive_field_size = tensor[0][0].numel()
        fan_in = num_input_fmaps * receptive_field_size
        fan_out = num_output_fmaps * receptive_field_size

    return fan_in, fan_out

In [207]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
       # self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
      #  x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [208]:
model = Net()

In [209]:
import math

In [210]:
def weights_init(m, xavier=True, param=10.):
    classname = m.__class__.__name__
    print (classname)
    if classname.find('Conv') != -1:
       # m.weight.data.normal_(0.0, 0.02)
        
        if xavier:
            z = _calculate_fan_in_and_fan_out(m.weight.data)
            std = math.sqrt(2.0 / (z[0] + z[1]))
            m.weight.data.uniform_(0.0, std)
        else:
            m.weight.data.normal_(0.0, 1.0)
    if classname.find('Linear') != -1:
        if xavier:
            z = _calculate_fan_in_and_fan_out(m.weight.data)
            std = math.sqrt(2.0 / (z[0] + z[1]))
            m.weight.data.uniform_(0.0, std)
            
            m.bias.data.fill_(0.0)
        else:
            m.weight.data.normal_(0.0, 1.0)
            m.bias.data.normal_
            m.weight.data.fill_(-1)
            m.bias.data.fill_(-10)



In [211]:
model.apply(weights_init)

Conv2d
Conv2d
Linear
Linear
Net


Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [212]:
args['lr'] = 0.01
args['momentum'] = 0.0

In [213]:
args['cuda'] = False

In [214]:
optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])

In [215]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

    model.eval()
    train_loss = 0
    correct = 0
    for data, target in train_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        train_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    train_loss /= len(train_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))

In [216]:
for epoch in range(1, 50 + 1):
    train(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 37.974968
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.302832
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.301864
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.303018
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.302191
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.302680
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.302471
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.302501
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.302693
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.301928
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.300943
Train Epoch: 1 [11000/60000 (18%)]	Loss: 2.302522
Train Epoch: 1 [12000/60000 (20%)]	Loss: 2.303563
Train Epoch: 1 [13000/60000 (22%)]	Loss: 2.301887
Train Epoch: 1 [14000/60000 (23%)]	Loss: 2.303252
Train Epoch: 1 [15000/60000 (25%)]	Loss: 2.303046
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2.302683
Train Epoch: 1 [17000/60000 (28%)]	Loss: 2.300770
Train Epoch: 1 [18000/60000 (30%)]	Loss: 2.302534
Train Epoch: 1 [19000/60000 (32%)]	Loss: 2.294522
Train Epoch: 1 [20

Train Epoch: 3 [43000/60000 (72%)]	Loss: 1.682094
Train Epoch: 3 [44000/60000 (73%)]	Loss: 1.403120
Train Epoch: 3 [45000/60000 (75%)]	Loss: 1.355701
Train Epoch: 3 [46000/60000 (77%)]	Loss: 1.470170
Train Epoch: 3 [47000/60000 (78%)]	Loss: 1.464707
Train Epoch: 3 [48000/60000 (80%)]	Loss: 1.464915
Train Epoch: 3 [49000/60000 (82%)]	Loss: 1.226624
Train Epoch: 3 [50000/60000 (83%)]	Loss: 1.473872
Train Epoch: 3 [51000/60000 (85%)]	Loss: 1.443268
Train Epoch: 3 [52000/60000 (87%)]	Loss: 1.443708
Train Epoch: 3 [53000/60000 (88%)]	Loss: 1.283865
Train Epoch: 3 [54000/60000 (90%)]	Loss: 1.375083
Train Epoch: 3 [55000/60000 (92%)]	Loss: 1.416502
Train Epoch: 3 [56000/60000 (93%)]	Loss: 1.303685
Train Epoch: 3 [57000/60000 (95%)]	Loss: 1.330994
Train Epoch: 3 [58000/60000 (97%)]	Loss: 1.214978
Train Epoch: 3 [59000/60000 (98%)]	Loss: 1.399117

Test set: Average loss: 1.3355, Accuracy: 33697/60000 (56%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.376600
Train Epoch: 4 [1000/60000 (2%)]	Loss: 1.40

Train Epoch: 6 [25000/60000 (42%)]	Loss: 0.634906
Train Epoch: 6 [26000/60000 (43%)]	Loss: 0.587689
Train Epoch: 6 [27000/60000 (45%)]	Loss: 0.595910
Train Epoch: 6 [28000/60000 (47%)]	Loss: 0.731779
Train Epoch: 6 [29000/60000 (48%)]	Loss: 0.464269
Train Epoch: 6 [30000/60000 (50%)]	Loss: 0.556224
Train Epoch: 6 [31000/60000 (52%)]	Loss: 0.601478
Train Epoch: 6 [32000/60000 (53%)]	Loss: 0.812522
Train Epoch: 6 [33000/60000 (55%)]	Loss: 0.729513
Train Epoch: 6 [34000/60000 (57%)]	Loss: 0.570522
Train Epoch: 6 [35000/60000 (58%)]	Loss: 0.723361
Train Epoch: 6 [36000/60000 (60%)]	Loss: 0.509404
Train Epoch: 6 [37000/60000 (62%)]	Loss: 0.642995
Train Epoch: 6 [38000/60000 (63%)]	Loss: 0.650629
Train Epoch: 6 [39000/60000 (65%)]	Loss: 0.726939
Train Epoch: 6 [40000/60000 (67%)]	Loss: 0.569543
Train Epoch: 6 [41000/60000 (68%)]	Loss: 0.632711
Train Epoch: 6 [42000/60000 (70%)]	Loss: 0.515577
Train Epoch: 6 [43000/60000 (72%)]	Loss: 0.568412
Train Epoch: 6 [44000/60000 (73%)]	Loss: 0.449214


Train Epoch: 9 [7000/60000 (12%)]	Loss: 0.248464
Train Epoch: 9 [8000/60000 (13%)]	Loss: 0.246224
Train Epoch: 9 [9000/60000 (15%)]	Loss: 0.359392
Train Epoch: 9 [10000/60000 (17%)]	Loss: 0.206863
Train Epoch: 9 [11000/60000 (18%)]	Loss: 0.399275
Train Epoch: 9 [12000/60000 (20%)]	Loss: 0.265924
Train Epoch: 9 [13000/60000 (22%)]	Loss: 0.378394
Train Epoch: 9 [14000/60000 (23%)]	Loss: 0.229378
Train Epoch: 9 [15000/60000 (25%)]	Loss: 0.443525
Train Epoch: 9 [16000/60000 (27%)]	Loss: 0.319040
Train Epoch: 9 [17000/60000 (28%)]	Loss: 0.234723
Train Epoch: 9 [18000/60000 (30%)]	Loss: 0.355311
Train Epoch: 9 [19000/60000 (32%)]	Loss: 0.349724
Train Epoch: 9 [20000/60000 (33%)]	Loss: 0.399624
Train Epoch: 9 [21000/60000 (35%)]	Loss: 0.201638
Train Epoch: 9 [22000/60000 (37%)]	Loss: 0.277345
Train Epoch: 9 [23000/60000 (38%)]	Loss: 0.280970
Train Epoch: 9 [24000/60000 (40%)]	Loss: 0.234829
Train Epoch: 9 [25000/60000 (42%)]	Loss: 0.164563
Train Epoch: 9 [26000/60000 (43%)]	Loss: 0.308590
Tra

KeyboardInterrupt: 

KeyError: 'lr'